<a href="https://colab.research.google.com/github/jgdshkovi/Benn/blob/master/BirchCos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/My Drive/Z/filter_pruning

/content/drive/My Drive/Z/filter_pruning


In [0]:
import argparse
import numpy as np
from numpy import unique
from numpy import where
import os

import math
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms

from models import *
#import spherical_kmeans_old as skm

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.cluster import Birch
import seaborn as sns
sns.set()

from matplotlib import pyplot
from matplotlib import pyplot as plt

cos = nn.CosineSimilarity()

In [0]:
test_batch_size = 256
dataset = 'cifar10'
cfg = [32, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 256, 256, 256, 'M', 256, 256, 256]
filtered_cfg = [16, 32, 'M', 64, 64, 'M', 128, 128, 128, 'M', 128, 128, 128, 'M', 128, 128, 128]
cuda = True

In [0]:
model = torch.load('Main_logs/model_best.pth.tar')
checkpoint = torch.load('Main_logs/checkpoint.pth.tar')

In [5]:
model = vgg(dataset='cifar10', depth=16)  #,cfg=cfg)
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [0]:
def test(model):
    kwargs = {'num_workers': 1, 'pin_memory': True} if True else {}
    if dataset == 'cifar10':
        test_loader = torch.utils.data.DataLoader(
            datasets.CIFAR10('./data.cifar10', train=False, transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])),
            batch_size=test_batch_size, shuffle=True, **kwargs)
    elif dataset == 'cifar100':
        test_loader = torch.utils.data.DataLoader(
            datasets.CIFAR100('./data.cifar100', train=False, transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])),
            batch_size=test_batch_size, shuffle=True, **kwargs)
    else:
        raise ValueError("No valid dataset is given.")
    model.eval()
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    print('\nTest set: Accuracy: {}/{} ({:.1f}%)\n'.format(
        correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))
    return correct / float(len(test_loader.dataset))

In [0]:
from collections import defaultdict

def list_duplicates(seq):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        tally[item].append(i)
    return ((key,locs) for key,locs in tally.items() if len(locs)>1)


def return_mask(wts, labels, thres):
	modsource = labels.copy()
	#print(wts)
	#print(labels)
	for dup in sorted(list_duplicates(labels)):
	    #print(dup)
	    lis = dup[1][1:]
	    a = torch.from_numpy(np.reshape(wts[dup[1][0]],(1,wts[dup[1][0]].size)))
	    #a = wts[dup[1][0]]
	    for i in lis:
	    	b = torch.from_numpy(np.reshape(wts[i],(1,wts[i].size)))
	    	#print
	    	simi = (cos(a,b))
	    	if simi>thres:
	    	  modsource[i] = -1

	mask = []
	for el in modsource:
		if el!=-1:
			mask.append(1)
		else:
			mask.append(0)
	return mask

def calc_distance(x1, y1, a, b, c):
  d = abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))
  return d

ress = []

def optimumk(X, shp):
  K = list(range(1,shp//2+1))
  K.append(shp)
  dist_points_from_cluster_center = []
  for no_of_clusters in K:
    res = skm.spherical_k_means(X,n_clusters=no_of_clusters,random_state=10)
    ress.append(res[1])
    dist_points_from_cluster_center.append(res[2])

  a = dist_points_from_cluster_center[0] - dist_points_from_cluster_center[shp//2]
  b = K[shp//2] - K[0]
  c1 = K[0] * dist_points_from_cluster_center[shp//2]
  c2 = K[shp//2] * dist_points_from_cluster_center[0]
  c = c1 - c2

  distance_of_points_from_line = []
  for k in range(shp//2+1):
    distance_of_points_from_line.append(
        calc_distance(K[k], dist_points_from_cluster_center[k], a, b, c))
  
  optk = distance_of_points_from_line.index(max(distance_of_points_from_line))
  return optk


def return_cluster_labels(X, shp,thres,bf):  #no_of_clus):
  brc = Birch(branching_factor=bf, n_clusters=None, threshold=thres)
  brc.fit(X)
  labels = brc.predict(X)
  return labels

In [0]:
def formclus(X, noc):
  mod = KMeans(n_clusters=noc)
  mod.fit(X)
  yhat = mod.predict(X)
  clusters = unique(yhat)
  for cluster in clusters:
    row_ix = where(yhat == cluster)
    pyplot.scatter(X[row_ix, 0], X[row_ix, 1])

In [0]:
def applyTSNE(X):
  shp = X.shape[1]
  if shp<50:
    noc = shp
  else:
    noc = 50
  pca50 = PCA(n_components=noc)
  pca50res = pca50.fit_transform(X)
  fashion_pca_tsne = TSNE(random_state=10).fit_transform(pca50res)
  xs = fashion_pca_tsne[:,0]
  ys = fashion_pca_tsne[:,1]

  Y = []
  for x,y in zip(xs,ys):
	  Y.append([x,y])
  Y = np.array(Y)
  return Y

  #find opt k here
  formclus(Y, noc=k)
  plt.show()

In [32]:
cos_cfg = []
cfg_mask = []
layer_id = 0

COS_THRES = 0.1
BIR_THRES = 0.1
BF = 50
while COS_THRES<0.7:
    BIR_THRES = 0.25
    cos_cfg = []
    for m in model.modules():
      if isinstance(m , nn.Conv2d):
        shape = m.weight.data.shape
        #print(shape)
        reshaped_tensor = m.weight.data.clone().numpy().reshape(shape[0] , shape[1]*shape[2]*shape[3])

        ##Y = applyTSNE(reshaped_tensor)

        labels = return_cluster_labels(reshaped_tensor,shape[0],thres=BIR_THRES,bf=50)  #filtered_cfg[layer_id])
        mask = return_mask(reshaped_tensor, labels, thres=COS_THRES)
        #print(mask)
        ##k = len(set(labels))
        #print(len(set(labels)))
        #print(sum(mask))
        cos_cfg.append(sum(mask))
        cfg_mask.append(torch.tensor(mask))

        ##formclus(Y, noc=k)
        ##plt.show()
            
        layer_id += 1
        #break
      elif isinstance(m, nn.MaxPool2d):
        layer_id += 1
        cos_cfg.append('M')
      #if layer_id>3:
        #break
    print('COS_THRES, BIR_THRES --- ',end='')
    print(COS_THRES, BIR_THRES)
    print(cos_cfg)
    
    COS_THRES += 0.05
    print()

COS_THRES, BIR_THRES --- 0.1 0.25
[41, 63, 'M', 128, 128, 'M', 256, 256, 256, 'M', 483, 318, 299, 'M', 299, 352, 308]

COS_THRES, BIR_THRES --- 0.15000000000000002 0.25
[41, 63, 'M', 128, 128, 'M', 256, 256, 256, 'M', 483, 321, 331, 'M', 347, 386, 345]

COS_THRES, BIR_THRES --- 0.2 0.25
[41, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 483, 328, 352, 'M', 385, 412, 364]

COS_THRES, BIR_THRES --- 0.25 0.25
[43, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 483, 338, 392, 'M', 420, 432, 381]

COS_THRES, BIR_THRES --- 0.3 0.25
[43, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 485, 346, 405, 'M', 443, 455, 395]

COS_THRES, BIR_THRES --- 0.35 0.25
[44, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 485, 356, 422, 'M', 471, 471, 406]

COS_THRES, BIR_THRES --- 0.39999999999999997 0.25
[47, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 485, 372, 439, 'M', 487, 482, 413]

COS_THRES, BIR_THRES --- 0.44999999999999996 0.25
[49, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 489, 387, 447, 'M', 498, 487, 427]

In [0]:
cos thres = 0.5
Main,bf=50
[64, 64, 'M', 20, 20, 'M', 45, 55, 38, 'M', 241, 236, 213, 'M', 255, 235, 209]---Y,0.9
[64, 64, 'M', 128, 29, 'M', 86, 80, 256, 'M', 339, 345, 332, 'M', 372, 380, 333]---Y,0.5
[49, 63, 'M', 127, 128, 'M', 256, 256, 256, 'M', 512, 509, 509, 'M', 502, 495, 435]---0.9
[49, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 490, 501, 509, 'M', 502, 495, 435]---0.5
[49, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 492, 408, 462, 'M', 502, 495, 435]---0.25
[50, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 497, 485, 'M', 316, 264, 219]---0.1

L1,bf=50
[32, 64, 'M', 17, 17, 'M', 52, 57, 256, 'M', 114, 121, 106, 'M', 104, 95, 81]      ---Y,0.9
[32, 64, 'M', 128, 128, 'M', 93, 78, 256, 'M', 179, 178, 163, 'M', 174, 171, 147]  ---Y,0.5
[28, 62, 'M', 128, 128, 'M', 256, 256, 256, 'M', 256, 253, 252, 'M', 250, 240, 214]---  1.5
[28, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 248, 252, 252, 'M', 250, 240, 214]---  0.5

In [0]:
print(cos_cfg)

[56, 62, 'M', 128, 128, 'M', 256, 256, 256, 'M', 510, 438, 417, 'M', 173, 160, 169]


In [0]:
cos = nn.CosineSimilarity()

In [0]:
a = torch.from_numpy(np.reshape(X[0],(1,X[0].size)))
b = torch.from_numpy(np.reshape(X[5],(1,X[5].size)))


In [0]:
newmodel = vgg(dataset = 'cifar10' ,cfg=cos_cfg)
newmodel.cuda()

start_mask = torch.ones(3)
layer_id_in_cfg = 0
end_mask = cfg_mask[layer_id_in_cfg]
for [m0, m1] in zip(model.modules(), newmodel.modules()):
    if isinstance(m0, nn.BatchNorm2d):
        idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
        if idx1.size == 1:
            idx1 = np.resize(idx1,(1,))
        m1.weight.data = m0.weight.data[idx1.tolist()].clone()
        m1.bias.data = m0.bias.data[idx1.tolist()].clone()
        m1.running_mean = m0.running_mean[idx1.tolist()].clone()
        m1.running_var = m0.running_var[idx1.tolist()].clone()
        layer_id_in_cfg += 1
        start_mask = end_mask
        if layer_id_in_cfg < len(cfg_mask):  # do not change in Final FC
            end_mask = cfg_mask[layer_id_in_cfg]
    elif isinstance(m0, nn.Conv2d):
        idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
        idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
        print('In shape: {:d}, Out shape {:d}.'.format(idx0.size, idx1.size))
        if idx0.size == 1:
            idx0 = np.resize(idx0, (1,))
        if idx1.size == 1:
            idx1 = np.resize(idx1, (1,))
        w1 = m0.weight.data[:, idx0.tolist(), :, :].clone()
        w1 = w1[idx1.tolist(), :, :, :].clone()
        m1.weight.data = w1.clone()
    elif isinstance(m0, nn.Linear):
        if layer_id_in_cfg == len(cfg_mask):
            idx0 = np.squeeze(np.argwhere(np.asarray(cfg_mask[-1].cpu().numpy())))
            if idx0.size == 1:
                idx0 = np.resize(idx0, (1,))
            m1.weight.data = m0.weight.data[:, idx0].clone()
            m1.bias.data = m0.bias.data.clone()
            layer_id_in_cfg += 1
            continue
        m1.weight.data = m0.weight.data.clone()
        m1.bias.data = m0.bias.data.clone()
    elif isinstance(m0, nn.BatchNorm1d):
        m1.weight.data = m0.weight.data.clone()
        m1.bias.data = m0.bias.data.clone()
        m1.running_mean = m0.running_mean.clone()
        m1.running_var = m0.running_var.clone()

In shape: 3, Out shape 56.
In shape: 56, Out shape 62.
In shape: 62, Out shape 128.
In shape: 128, Out shape 128.
In shape: 128, Out shape 256.
In shape: 256, Out shape 256.
In shape: 256, Out shape 256.
In shape: 256, Out shape 510.
In shape: 510, Out shape 438.
In shape: 438, Out shape 417.
In shape: 417, Out shape 173.
In shape: 173, Out shape 160.
In shape: 160, Out shape 169.


In [0]:
torch.save({'cfg': cos_cfg, 'state_dict': newmodel.state_dict()},'Xmeans_logs/0.4/pruned.pth.tar')
print(newmodel)
model = newmodel
model.cuda()
acc = test(model)

num_parameters = sum([param.nelement() for param in newmodel.parameters()])
with open( "Xmeans_logs/0.4/prune.txt", "w") as fp:
    fp.write("Number of parameters: \n"+str(num_parameters)+"\n")
    fp.write("Test accuracy: \n"+str(acc)+"\n")

vgg(
  (feature): Sequential(
    (0): Conv2d(3, 56, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(56, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(56, 62, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(62, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(62, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilatio

In [0]:
!python main_finetune.py --refine Filter_logs/pruned.pth.tar --dataset cifar10 --epochs 10 --save Filter_logs/

Files already downloaded and verified
Train Epoch: 0 [0/50000 (0.0%)]	Loss: 3.237350
Train Epoch: 0 [6400/50000 (12.8%)]	Loss: 0.726783
Train Epoch: 0 [12800/50000 (25.6%)]	Loss: 0.547180
Train Epoch: 0 [19200/50000 (38.4%)]	Loss: 0.646843
Train Epoch: 0 [25600/50000 (51.2%)]	Loss: 0.271935
Train Epoch: 0 [32000/50000 (63.9%)]	Loss: 0.413913
Train Epoch: 0 [38400/50000 (76.7%)]	Loss: 0.418347
Train Epoch: 0 [44800/50000 (89.5%)]	Loss: 0.522038
main_finetune.py:153: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True), Variable(target)
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))

Test set: Average loss: 0.4774, Accuracy: 8480/10000 (84.8%)

Train Epoch: 1 [0/50000 (0.0%)]	Loss: 0.342160
Train Epoch: 1 [6400/50000 (12.8%)]	Loss: 0.275073
Train Epo